In [101]:
from selenium import webdriver
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')
db = client.megasena

def megasena(concurso):
    options = webdriver.FirefoxOptions()
    options.add_argument('headless') # O mais importante, não renderiza parte gráfica
    options.add_argument('window-size=1920x1080')
    driver = webdriver.Firefox(executable_path = '/home/gustavo/Downloads/geckodriver-v0.29.0-linux32/geckodriver', options = options)
    url = 'https://www.loteriaseresultados.com.br/index.php/megasena/concurso/' + str(concurso)
    driver.get(url)
    try:
        # uma opção é obter todo o body (ou parte mais específica) da página com o selenium/webdriver e passar ao BSoup para análise padrão
        body = driver.find_element_by_tag_name('body')
        html = body.get_attribute('innerHTML')
        soupPage = soup(html, 'html5lib')
        spanList = soupPage.findAll('span', class_='white--text font-weight-bold')
        tdCenterList = soupPage.findAll('td', class_='text-center')
        tdRightList = soupPage.findAll('td', class_='text-right')
        dataList = []
#         print(spanList)
#         print(tdCenterList)
#         print(tdRightList)
        for n in range(6):
            print(spanList[n].text)
            dataList.append(spanList[n].text)
            
        print(dataList)
        db.concursos.insert_one({
            str(concurso): dataList,
            'SENA': {'Resultado': tdCenterList[1].text, 'Prêmio': tdRightList[0].text},
            'QUINA': {'Resultado': tdCenterList[3].text, 'Prêmio': tdRightList[1].text},
            'QUADRA': {'Resultado': tdCenterList[5].text, 'Prêmio': tdRightList[2].text}
        })
    
    except:
        print('criterio nao encontrado')
    
    print('Terminei a raspagem de dados')
    driver.close()

megasena(2336)

04
18
29
47
48
59
['04', '18', '29', '47', '48', '59']
Terminei a raspagem de dados
